In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format('csv')\
    .option('header',True)\
        .option('inferschema',True)\
            .load('/FileStore/RealTime/rawsource/sales_data_first.csv')

In [0]:
df.limit(5).display()

Branch_ID,Dealer_ID,Model_ID,Revenue,Units_Sold,Date_ID,Month,Year,BranchName,DealerName,Product_Name,Date
BR0006,DLR0168,Ren-M128,12971088,3,DT01236,5,2020,AC Cars Motors,Saab Motors,Renault,5/20/2020 0:00
BR0011,DLR0069,Vol-M256,14181510,3,DT01225,5,2020,Acura Motors,Geo Motors,Volkswagen,5/9/2020 0:00
BR0021,DLR0070,Vol-M257,7738896,1,DT01226,5,2020,Aixam-Mega (including Arola) Motors,Gilbern Motors,Volkswagen,5/10/2020 0:00
BR0031,DLR0071,Vol-M258,10067596,2,DT01227,5,2020,Alfa Romeo Motors,Ginetta Motors,Volkswagen,5/11/2020 0:00
BR0041,DLR0072,Vol-M259,13055810,2,DT01228,5,2020,Alpine Motors,Glas Motors,Volkswagen,5/12/2020 0:00


In [0]:
def conditioncol(x):
    if x==1:
        return "low"
    elif x==2:
        return "Medium"
    else:
        return "High"

In [0]:
condition = udf(conditioncol)

In [0]:
df=df.withColumn("ConditionalColumnNew",condition("Units_Sold"))

In [0]:
df.limit(5).display()

Branch_ID,Dealer_ID,Model_ID,Revenue,Units_Sold,Date_ID,Month,Year,BranchName,DealerName,Product_Name,Date,ConditionalColumnNew
BR0006,DLR0168,Ren-M128,12971088,3,DT01236,5,2020,AC Cars Motors,Saab Motors,Renault,5/20/2020 0:00,High
BR0011,DLR0069,Vol-M256,14181510,3,DT01225,5,2020,Acura Motors,Geo Motors,Volkswagen,5/9/2020 0:00,High
BR0021,DLR0070,Vol-M257,7738896,1,DT01226,5,2020,Aixam-Mega (including Arola) Motors,Gilbern Motors,Volkswagen,5/10/2020 0:00,low
BR0031,DLR0071,Vol-M258,10067596,2,DT01227,5,2020,Alfa Romeo Motors,Ginetta Motors,Volkswagen,5/11/2020 0:00,Medium
BR0041,DLR0072,Vol-M259,13055810,2,DT01228,5,2020,Alpine Motors,Glas Motors,Volkswagen,5/12/2020 0:00,Medium


##### Cummulative Sum 

In [0]:
windo_space=Window.partitionBy('Month').orderBy('Month').rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [0]:
df= df.withColumn('CumSum',sum(col('Units_Sold')*col('Revenue')).over(windo_space))

In [0]:
df.select('Month','Units_Sold','Revenue','CumSum').limit(5).display()

Month,Units_Sold,Revenue,CumSum
1,1,946626,946626
1,3,2349414,7994868
1,2,5501066,18997000
1,2,15299630,49596260
1,1,573260,50169520
